In [1]:
import pandas as pd
import numpy as np

from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

from sklearn.metrics import accuracy_score

In [2]:
main_df = pd.read_csv("./data/winequality-white.csv", sep=";")
main_df.head(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6


In [3]:
main_df.describe() # high level overview of the dataset

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267,5.877909
std,0.843868,0.100795,0.121020,5.072058,0.021848,17.007137,42.498065,0.002991,0.151001,0.114126,1.230621,0.885639
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


In [18]:
main_df.isnull().any() # checking if any column has null values

fixed acidity           False
volatile acidity        False
citric acid             False
residual sugar          False
chlorides               False
free sulfur dioxide     False
total sulfur dioxide    False
density                 False
pH                      False
sulphates               False
alcohol                 False
quality                 False
dtype: bool

In [5]:
main_df["quality"] -= main_df["quality"].min()
main_df["quality"].value_counts()

3    2198
2    1457
4     880
5     175
1     163
0      20
6       5
Name: quality, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split

X = main_df.drop(columns=["quality"], inplace=False)
y = main_df["quality"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [7]:
normalizer = StandardScaler()

I will be using the guide on https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html for selecting the algorithms.

In [8]:
svm = LinearSVC(dual=False) # dual=True is the default, "Prefer dual=False when n_samples > n_features" as stated in scikit-learn docs.
svm.fit(X_train, y_train)

svm.score(X_test, y_test)

0.5142857142857142

In [9]:
normalized_svm = make_pipeline(normalizer, LinearSVC(dual=False))
normalized_svm.fit(X_train, y_train)

normalized_svm.score(X_test, y_test)

0.5061224489795918

In [10]:
knn = KNeighborsClassifier(n_neighbors=7, weights="distance")
knn.fit(X_train, y_train)

knn.score(X_test, y_test)

0.5948979591836735

In [11]:
normalized_knn = make_pipeline(normalizer, KNeighborsClassifier(n_neighbors=7, weights="distance"))
normalized_knn.fit(X_train, y_train)

normalized_knn.score(X_test, y_test)

0.6581632653061225

In [12]:
xgb_model = XGBClassifier(use_label_encoder=False, max_depth=8)
xgb_model.fit(X_train, y_train)

xgb_predictions = xgb_model.predict(X_test)
accuracy_score(y_test, xgb_predictions)

[18:02:41] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.6918367346938775

In [16]:
normalized_xgb_model = make_pipeline(normalizer, XGBClassifier(use_label_encoder=False, max_depth=8))
normalized_xgb_model.fit(X_train, y_train)

normalized_xgb_predictions = normalized_xgb_model.predict(X_test)
accuracy_score(y_test, normalized_xgb_predictions)

[18:06:01] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.6908163265306122